In [1]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random
import itertools

import sys
import os
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import six
from abc import ABCMeta
from scipy import sparse
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.svm import LinearSVC

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
plt.style.use('ggplot')

Using TensorFlow backend.


In [184]:
data_file = './/Amazon_Unlocked_Mobile.csv'
n = 413000  
s = 3000
skip = sorted(random.sample(range(1,n),n-s))


df = pd.read_csv( data_file, delimiter = ",", skiprows = skip)
#41만개의 데이터 중 3000개의 데이터만 뽑아서 분석을 진행 

In [185]:
df

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,[XMAS DEAL] [New Edition] Jethro [SC213V2] Fli...,Jethro,79.99,2,does not work well with T-Mobile. T-Mobile clo...,1.0
1,10.6'' Inch E-passion Unlocked Smart Phone +Ta...,e passion,99.95,5,Excellent,0.0
2,"4 Inch Touch Screen Cell Phone Unlocked, Andro...",NaN,23.90,5,Better than I expected.,0.0
3,"5.0"" Elephone P6000 MTK6732 64-bit Quad Core 2...",Elephone,161.06,5,It's spectacular. This phone hace a good perfo...,2.0
4,"5.5"" HD Display Unlocked Android Smartphones 4...",Plum,99.99,1,Phone is cheap...only had it for 3-5 months n ...,0.0
...,...,...,...,...,...,...
3835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,another great deal great price,0.0
3836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,Ok,0.0
3837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,Passes every drop test onto porcelain tile!,0.0
3838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,I returned it because it did not meet my needs...,0.0


In [186]:
df.isnull().sum()

Product Name      0
Brand Name      508
Price            45
Rating            0
Reviews           1
Review Votes     99
dtype: int64

In [187]:
df = df.dropna()

In [188]:
df.isnull().sum()

Product Name    0
Brand Name      0
Price           0
Rating          0
Reviews         0
Review Votes    0
dtype: int64

In [189]:
df
#결측치들을 제거하니 41만개의 데이터에서 33만개의 데이터로 축소되었음
#아마 하나의 row에 결측치가 한개라도 존재하면 해당 row를 제거하는 방법을 수행했을것으로 예상 

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,[XMAS DEAL] [New Edition] Jethro [SC213V2] Fli...,Jethro,79.99,2,does not work well with T-Mobile. T-Mobile clo...,1.0
1,10.6'' Inch E-passion Unlocked Smart Phone +Ta...,e passion,99.95,5,Excellent,0.0
3,"5.0"" Elephone P6000 MTK6732 64-bit Quad Core 2...",Elephone,161.06,5,It's spectacular. This phone hace a good perfo...,2.0
4,"5.5"" HD Display Unlocked Android Smartphones 4...",Plum,99.99,1,Phone is cheap...only had it for 3-5 months n ...,0.0
12,"6.0"" FWVGA Screen MTK6577 Dual Core 1.2GHz And...",Ulefone,69.99,3,The phone ring volume is so low even on high a...,0.0
...,...,...,...,...,...,...
3835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,another great deal great price,0.0
3836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,Ok,0.0
3837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,Passes every drop test onto porcelain tile!,0.0
3838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,I returned it because it did not meet my needs...,0.0


### Rating Mapping

In [190]:
Review_mapping =  {0: '0',1 : '0', 2 :'0',3 : '0', 4 : '1', 5 : '1'}
Review_mapping
#0~3점까지는 0으로 4~5점까지는 1로 맵핑 

{0: '0', 1: '0', 2: '0', 3: '0', 4: '1', 5: '1'}

In [191]:
Rating = lambda x: Review_mapping.get(x,x)
df['Rating']=df.Rating.map(Rating)

/Users/Moon/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [192]:
df['Rating'].unique()

array(['0', '1'], dtype=object)

In [193]:
df
#rating이 잘변환된것을 확인할 수 있다.

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,[XMAS DEAL] [New Edition] Jethro [SC213V2] Fli...,Jethro,79.99,0,does not work well with T-Mobile. T-Mobile clo...,1.0
1,10.6'' Inch E-passion Unlocked Smart Phone +Ta...,e passion,99.95,1,Excellent,0.0
3,"5.0"" Elephone P6000 MTK6732 64-bit Quad Core 2...",Elephone,161.06,1,It's spectacular. This phone hace a good perfo...,2.0
4,"5.5"" HD Display Unlocked Android Smartphones 4...",Plum,99.99,0,Phone is cheap...only had it for 3-5 months n ...,0.0
12,"6.0"" FWVGA Screen MTK6577 Dual Core 1.2GHz And...",Ulefone,69.99,0,The phone ring volume is so low even on high a...,0.0
...,...,...,...,...,...,...
3835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,1,another great deal great price,0.0
3836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,0,Ok,0.0
3837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,1,Passes every drop test onto porcelain tile!,0.0
3838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,0,I returned it because it did not meet my needs...,0.0


### 텍스트 데이터 분리

In [194]:
df.columns

Index(['Product Name', 'Brand Name', 'Price', 'Rating', 'Reviews',
       'Review Votes'],
      dtype='object')

In [195]:
a = df.iloc[:,3]
print(a)
# a는 평점
a.astype('int64')
#맵핑을 한 rating이 object로 되어있어 int로 변환을 하였음 

0       0
1       1
3       1
4       0
12      0
       ..
3835    1
3836    0
3837    1
3838    0
3839    1
Name: Rating, Length: 3222, dtype: object


0       0
1       1
3       1
4       0
12      0
       ..
3835    1
3836    0
3837    1
3838    0
3839    1
Name: Rating, Length: 3222, dtype: int64

In [196]:
b = df.iloc[:,4]
# b는 리뷰데이터(텍스트)

In [197]:
doc = pd.concat([a,b], axis = 1)
#기존의 데이터에서 rating과 review로 구성된 새로운 데이터셋을 구성 

### Text Cleaning

In [198]:
def review_to_wordlist(review, remove_stopwords = True):

    
    review_text = re.sub('[^a-zA-Z]'," ", review)
    #review_text에 영어만 넣기 
    
    words = review_text.lower().split()
    #소문자로 바꿔주고 그것들을 분리해준다 
    
    if remove_stopwords:
        stops = set(stopwords.words('english'))
        words = [w for w in words if not w in stops]
        #stops에 영어의 불용어를 넣어줌 
        #words는 소문자로 변환되고 띄어져있는 것이며 
        #stops에 있는 불용어를 제외하고 넣어줌 
        
    b = []
    stemmer = english_stemmer
    for word in words:
        b.append(stemmer.stem(word))
        #words에 전처리된것들의 어간들만 추출 
        
    return(b)


In [199]:
clean_reviews = []
for review in doc['Reviews']:
    clean_reviews.append( " ".join(review_to_wordlist(review)))

In [200]:
len(clean_reviews)
#각 row마다 텍스트 전처리가 잘된것을 확인 

3222

In [202]:
clean_reviews = pd.DataFrame(clean_reviews)

In [203]:
clean_reviews.columns = ['Reviews']
clean_reviews
#컬럼명을 Reviews로 설정 

,Reviews
0,work well mobil mobil close tower non mobil phone
1,excel
2,spectacular phone hace good perfom like mucho ...
3,phone cheap month n alreadi stop work could do...
4,phone ring volum low even high case fit phone ...
...,...
3217,anoth great deal great price
3218,ok
3219,pass everi drop test onto porcelain tile
3220,return meet need seem good select other


In [204]:
df = pd.concat([clean_reviews, doc['Rating']], axis = 1)
df.isnull().sum()
#무작정 concat을 사용했기때문에 결측값이 발생 
#index를 맞춰주지않아 문제가 발생한 것으로 보인다.

Reviews    602
Rating     602
dtype: int64

In [205]:
clean_reviews.reset_index(drop=True , inplace=True)
clean_reviews
#인덱스를 초기화해줌 

,Reviews
0,work well mobil mobil close tower non mobil phone
1,excel
2,spectacular phone hace good perfom like mucho ...
3,phone cheap month n alreadi stop work could do...
4,phone ring volum low even high case fit phone ...
...,...
3217,anoth great deal great price
3218,ok
3219,pass everi drop test onto porcelain tile
3220,return meet need seem good select other


In [206]:
b = doc['Rating']

In [207]:
b.reset_index(drop = True, inplace = True)
b
#rating의 인덱스를 초기화해줌 

0       0
1       1
2       1
3       0
4       0
       ..
3217    1
3218    0
3219    1
3220    0
3221    1
Name: Rating, Length: 3222, dtype: object

In [208]:
Data = pd.concat([clean_reviews, doc['Rating']], axis = 1)
Data.isnull().sum()
#인덱스를 초기화하여 각각의 데이터프레임을 결합을 하였으며 결측값이 발생하지 않음 
#인덱스를 초기화해주지않아서 문제가 발생을 하였다

Reviews    0
Rating     0
dtype: int64

In [209]:
Data

,Reviews,Rating
0,work well mobil mobil close tower non mobil phone,0
1,excel,1
2,spectacular phone hace good perfom like mucho ...,1
3,phone cheap month n alreadi stop work could do...,0
4,phone ring volum low even high case fit phone ...,0
...,...,...
3217,anoth great deal great price,1
3218,ok,0
3219,pass everi drop test onto porcelain tile,1
3220,return meet need seem good select other,0


In [210]:
train_docs, test_docs = train_test_split(Data, test_size = 0.3)
#학습과 테스트를 위해 원본 데이터를 7:3비율로 분할 

### TF-IDF

In [211]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
NGRAM_RANGE = (1,2)
TOP_K = 2000
TOKEN_MODE = 'word'
MIN_DOC_FREQ = 2

In [212]:
def ngram_vectorize(texts, labels):
    kwargs = {
        'ngram_range' : NGRAM_RANGE,
        'dtype' : 'int32',
        'strip_accents' : 'unicode',
        'decode_error' : 'replace',
        'analyzer' : TOKEN_MODE,
        'min_df' : MIN_DOC_FREQ
    }
    
    tfidf_vectorizer = TfidfVectorizer(**kwargs)
    transformed_texts = tfidf_vectorizer.fit_transform(texts)
    
    selector = SelectKBest(f_classif, k = min(TOP_K, transformed_texts.shape[1]))
    selector.fit(transformed_texts, labels)
    
    transformed_texts = selector.transform(transformed_texts).astype('float32')
    return transformed_texts
#TF -IDF를 변환하기 위한 각각의 파라미터를 조정하였음 

In [216]:
vect_train_data = ngram_vectorize(train_docs['Reviews'], train_docs['Rating'])
vect_train_data

/Users/Moon/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1616: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


<2255x2000 sparse matrix of type '<class 'numpy.float32'>'
	with 27418 stored elements in Compressed Sparse Row format>

In [217]:
vect_test_data = ngram_vectorize(test_docs['Reviews'], test_docs['Rating'])
vect_test_data

/Users/Moon/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1616: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


<967x2000 sparse matrix of type '<class 'numpy.float32'>'
	with 13077 stored elements in Compressed Sparse Row format>

In [218]:
rf_X_train = vect_train_data.toarray()
rf_y_train = (np.array(train_docs['Rating']))
rf_X_test = vect_test_data.toarray()
rf_y_test = (np.array(test_docs['Rating']))

### Random Forest

In [219]:
from sklearn.ensemble import RandomForestClassifier

In [220]:
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=2020)
forest 
#데이터가 너무 커서 최적의 파라미터들을 찾기보다는 default로 두는 것이 가장 좋은 선택이라는 판단 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=2020, verbose=0,
                       warm_start=False)

In [221]:
forest = forest.fit(rf_X_train, rf_y_train)

In [222]:
result = forest.predict(rf_X_test)

In [223]:
rf_y_test = rf_y_test.astype('int64')

In [225]:
result = result.astype('int64')

In [226]:
def get_clf_eval(y_test, preds):
    confusion = confusion_matrix(y_test, preds)
    accuracy = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    F1 = f1_score(y_test, preds)
    AUC = roc_auc_score(y_test, preds)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))
    #Confusion Matrix, Accuracy, precision, recall, F1, AUC등을 한번에 보여주는 함수를 생성 

In [227]:
get_clf_eval(rf_y_test, result)
#랜덤포레스트모델에서는 재현율은 아주 좋은 성능을 보여주며 f1 score는 재현율에 영향을 받기때문에 높은 값으로 나온것롤 보인다 
#하지만 다른 성능지표에서는 그렇게 좋지 못한 성능을 보여주고 있다.

오차행렬:
 [[ 56 242]
 [ 54 615]]

정확도: 0.6939
정밀도: 0.7176
재현율: 0.9193
F1: 0.8060
AUC: 0.5536


### XGBOOST

In [326]:
import xgboost as xgb 
from xgboost import plot_importance 
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from xgboost import XGBClassifier

In [327]:
dtrain = xgb.DMatrix(data = rf_X_train, label = rf_y_train)
dtest = xgb.DMatrix(data= rf_X_test, label = rf_y_test)

#numpy형태의 학습 데이터셋과 테스트 데이터셋을 DMatirx로 변환
#xgboost 모델을 학습시키기위해서는 Dmatrix로 변환을 해야하는것 같음 

In [328]:
params = {'max_depth' : 4,
          'eta' : 0.1,
          'objective' : 'binary:logistic',
          'eval_metric' : 'logloss',
          'early_stoppings' : 30}

num_rounds = 100
#eta : 학습률 
#손실함수 평가지표 : logloss
#파라미터를 

In [329]:
wlist = [(dtrain, 'train'), (dtest, 'eval')]
xgb_model = xgb.train(params = params, 
                      dtrain = dtrain, 
                      num_boost_round = num_rounds,
                      evals = wlist)

[13:04:16] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { early_stoppings } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.66371	eval-logloss:0.68640
[1]	train-logloss:0.63909	eval-logloss:0.68041
[2]	train-logloss:0.61831	eval-logloss:0.66941
[3]	train-logloss:0.60042	eval-logloss:0.66270
[4]	train-logloss:0.58357	eval-logloss:0.66270
[5]	train-logloss:0.56980	eval-logloss:0.65563
[6]	train-logloss:0.55825	eval-logloss:0.65019
[7]	train-logloss:0.54613	eval-logloss:0.65111
[8]	train-logloss:0.53590	eval-logloss:0.64448
[9]	train-logloss:0.52593	eval-logloss:0.64576
[10]	train-logloss:0.51775	eval-logloss:0.64241
[11]	train-logloss:0.51028	eval-logloss:0.64236
[12]	train-logloss:0.50375	eval-logloss:0.63845
[13]	train-logloss:0.

In [362]:
pred_probs = xgb_model.predict(dtest)
#예측값을 확률로 출력 

In [356]:
preds = [1 if x > 0.4 else 0 for x in pred_probs]

In [357]:
def get_clf_eval(y_test, preds):
    confusion = confusion_matrix(y_test, preds)
    accuracy = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    F1 = f1_score(y_test, preds)
    AUC = roc_auc_score(y_test, preds)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))
#Confusion Matrix, Accuracy, precision, recall, F1, AUC등을 한번에 보여주는 함수를 생성 

In [358]:
get_clf_eval(rf_y_test, preds)
#재현율에서는 아주 높은 값을 보여주고 있지만 나머지값들에 대해서는 좋지 못한 결과를 보여준다
#또한 AUC가 0.5정도면 좋지 못한 모델이라는 것을 보여준다

오차행렬:
 [[  9 289]
 [ 10 659]]

정확도: 0.6908
정밀도: 0.6951
재현율: 0.9851
F1: 0.8151
AUC: 0.5076


### LSTM

In [236]:
X_train = rf_X_train
y_train = rf_y_train 
X_test = rf_X_test
y_test = rf_y_test

In [269]:
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [320]:
embed_size = 64
max_features = 1500


model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

#6개의 layer로 구성되어 있으며 성능을 좋게 내기 위해 학습 모델 코드를 참조하였음 

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, None, 64)          96000     
_________________________________________________________________
bidirectional_16 (Bidirectio (None, None, 64)          24832     
_________________________________________________________________
global_max_pooling1d_14 (Glo (None, 64)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 20)                1300      
_________________________________________________________________
dropout_16 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_43 (Dense)             (None, 1)                 21        
Total params: 122,153
Trainable params: 122,153
Non-trainable params: 0
_______________________________________________

In [359]:
batch_size = 256
epochs = 10
model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, validation_split=0.3)
#Validation set을 train셋에서 30%를 두고 학습을 진행 

Train on 1578 samples, validate on 677 samples
Epoch 1/10
1578/1578 [==============================] - 3s 2ms/step - loss: 0.6119 - accuracy: 0.7003 - val_loss: 0.6196 - val_accuracy: 0.6898
Epoch 2/10
1578/1578 [==============================] - 3s 2ms/step - loss: 0.6129 - accuracy: 0.7003 - val_loss: 0.6194 - val_accuracy: 0.6898
Epoch 3/10
1578/1578 [==============================] - 3s 2ms/step - loss: 0.6117 - accuracy: 0.7003 - val_loss: 0.6195 - val_accuracy: 0.6898
Epoch 4/10
1578/1578 [==============================] - 3s 2ms/step - loss: 0.6110 - accuracy: 0.7003 - val_loss: 0.6198 - val_accuracy: 0.6898
Epoch 5/10
1578/1578 [==============================] - 3s 2ms/step - loss: 0.6106 - accuracy: 0.7003 - val_loss: 0.6200 - val_accuracy: 0.6898
Epoch 6/10
1578/1578 [==============================] - 3s 2ms/step - loss: 0.6120 - accuracy: 0.7003 - val_loss: 0.6193 - val_accuracy: 0.6898
Epoch 7/10
1578/1578 [==============================] - 3s 2ms/step - loss: 0.6100 - accu

In [360]:
score, acc =model.evaluate(X_test, y_test, batch_size = batch_size)

967/967 [==============================] - 1s 623us/step


In [361]:
print('Test Score:' , score)
print('Test Accuracy:' , acc)
#69.18%의 정확도를 보여주고 있으며 좋지못한 성능을 보여준다 
#추후 성능을 올릴방법을 찾아서 성능을 올릴예정

Test Score: 0.6171910514382813
Test Accuracy: 0.6918303966522217
